# Capítulo 10 - Operações groupby: separar - aplicar - combinar

## Seção 10.2 - Agregação

A ideia da operação groupby é separar os dados originais em grupos e aplicar alguma função de sumarização em cada grupo, retornando um único valor para representar o grupo.

Essa é a ideia de agregação, ou seja, juntar vários valores e extrair um único valor deles (por exemplo, média, desvio padão, quantil etc). Outro nome para isso é sumarização.

Como exemplo, vamos calcular a média da expectativa de vida entre os diversos países da base de dados do gapminder agregados por ano:

In [1]:
import pandas as pd

df = pd.read_csv('../data/gapminder.tsv', sep='\t')

# Usando notação de ponto:
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
# Ou usando notação de colchetes:
avg_life_exp_by_year = df.groupby('year')['lifeExp'].mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


Há diversas funções de sumarização de dados sendo que uma muito interessant é a função describe(), que retorna no agrupamento o count, média, desvio padrão, min, max e percentis de 25%, 50% e 75%:

In [2]:
continent_describe = df.groupby('continent').lifeExp.describe()
print(continent_describe)

           count       mean        std     min       25%      50%       75%  \
continent                                                                     
Africa     624.0  48.865330   9.150210  23.599  42.37250  47.7920  54.41150   
Americas   300.0  64.658737   9.345088  37.579  58.41000  67.0480  71.69950   
Asia       396.0  60.064903  11.864532  28.801  51.42625  61.7915  69.50525   
Europe     360.0  71.903686   5.433178  43.585  69.57000  72.2410  75.45050   
Oceania     24.0  74.326208   3.795611  69.120  71.20500  73.6650  77.55250   

              max  
continent          
Africa     76.442  
Americas   80.653  
Asia       82.603  
Europe     81.757  
Oceania    81.235  


Além disso, podemos aplicar nossas próprias funções ao agrupamento:

In [3]:
def media_personalizada(valores):
    return valores.sum()/len(valores)

avg_life_exp_by_year_media_personalizada = df.groupby('year').lifeExp.agg(media_personalizada)
print(avg_life_exp_by_year_media_personalizada)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


Nossas funções podem ter parâmetros. Basta manter a série como o primeiro parâmetro:

In [4]:
def media_personalizada_com_vies(valores, vies):
    return valores.sum()/len(valores) + vies

avg_life_exp_by_year_media_personalizada_com_vies = df.groupby('year').lifeExp.agg(media_personalizada_com_vies, vies=-50)
print(avg_life_exp_by_year_media_personalizada_com_vies)

year
1952    -0.942380
1957     1.507401
1962     3.609249
1967     5.678290
1972     7.647386
1977     9.570157
1982    11.533197
1987    13.212613
1992    14.160338
1997    15.014676
2002    15.694923
2007    17.007423
Name: lifeExp, dtype: float64


É possível usar várias funções diferentes na mesma série:

In [5]:
import numpy as np

print(df.groupby('year').lifeExp.agg([np.mean, np.std]))

           mean        std
year                      
1952  49.057620  12.225956
1957  51.507401  12.231286
1962  53.609249  12.097245
1967  55.678290  11.718858
1972  57.647386  11.381953
1977  59.570157  11.227229
1982  61.533197  10.770618
1987  63.212613  10.556285
1992  64.160338  11.227380
1997  65.014676  11.559439
2002  65.694923  12.279823
2007  67.007423  12.073021


Podemos ainda usar dict para passar funções diferentes para sumarização de colunas diferentes. Observe que o dict aqui atua no dataframe:

In [6]:
print(
    df.groupby('year').agg(
    {
        'lifeExp': 'mean',
        'pop': ['median', 'mean'],
        'gdpPercap': 'median'
    })
)

        lifeExp         pop                  gdpPercap
           mean      median          mean       median
year                                                  
1952  49.057620   3943953.0  1.695040e+07  1968.528344
1957  51.507401   4282942.0  1.876341e+07  2173.220291
1962  53.609249   4686039.5  2.042101e+07  2335.439533
1967  55.678290   5170175.5  2.265830e+07  2678.334741
1972  57.647386   5877996.5  2.518998e+07  3339.129407
1977  59.570157   6404036.5  2.767638e+07  3798.609244
1982  61.533197   7007320.0  3.020730e+07  4216.228428
1987  63.212613   7774861.5  3.303857e+07  4280.300366
1992  64.160338   8688686.5  3.599092e+07  4386.085502
1997  65.014676   9735063.5  3.883947e+07  4781.825478
2002  65.694923  10372918.5  4.145759e+07  5319.804524
2007  67.007423  10517531.0  4.402122e+07  6124.371109


## Seção 10.3 - Transformação

Na agregação nós estamos retornando um valor para um grupo (n -> 1). No caso de uma transformação nós vamos pegar um valor e transformar em outro (n -> n).

Para isso usamos a função transform da série:

In [7]:
def transforma_zscore(x):
    return (x - x.mean())/x.std()

print(
    df.groupby('year').lifeExp.transform(transforma_zscore)
)

0      -1.656854
1      -1.731249
2      -1.786543
3      -1.848157
4      -1.894173
          ...   
1699   -0.081621
1700   -0.336974
1701   -1.574962
1702   -2.093346
1703   -1.948180
Name: lifeExp, Length: 1704, dtype: float64


No caso do transform seguido de groupby a função é aplicada em cada agrupamento. Se tivéssemos aplicado a função diretamente no dataframe o resultado seria outro, pois ele não teria separado em grupos. No exemplo abaixo, veja que os resultados são diferentes quando calculamos o z-score em todo o grupo de uma vez:

In [8]:
print(transforma_zscore(df.lifeExp))

print('O menor valor aplicando sobre toda a base é:')
print(transforma_zscore(df.lifeExp).min())
print('O menor valor quando o z-score é calculado em agrupamentos distintos é diferente:')
print(df.groupby('year').lifeExp.transform(transforma_zscore).min())

0      -2.374637
1      -2.256112
2      -2.127213
3      -1.970599
4      -1.810501
          ...   
1699    0.222694
1700    0.069873
1701   -0.980517
1702   -1.508499
1703   -1.237695
Name: lifeExp, Length: 1704, dtype: float64
O menor valor aplicando sobre toda a base é:
-2.7773585999499315
O menor valor quando o z-score é calculado em agrupamentos distintos é diferente:
-3.6127163896209087


O método transform também é útil para preencher valores na com informação do grupo. Por exemplo, vamos usar o dataframe tips da biblioteca seaborn e preencher os NaN da coluna total_bill com a média dos dados agrupados por gênero:

In [9]:
import seaborn as sns

# Lê 10 observações do databaset tips:
tips_10 = sns.load_dataset('tips').sample(10)

# Escolhe aleatoriamente 4 valores de total_bill e os transforma em NaN:
np.random.seed(30)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN

# Imprime
print(tips_10)

# Cria uma função que devolve a série com os valores de na preenchidos pela média da série:
def fill_na_mean(x):
    return x.fillna(x.mean())

# Aplica o fill_na_mean por grupo
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print('------------------------------------')
print(tips_10)
print('------------------------------------')
print('Observe acima que a coluna fill_total_bill possui valores diferentes de acordo com o sex, devido à aplicação ao agrupamento')

     total_bill   tip     sex smoker   day    time  size
113         NaN  2.55    Male     No   Sun  Dinner     2
207         NaN  3.00    Male    Yes   Sat  Dinner     4
183         NaN  6.50    Male    Yes   Sun  Dinner     4
63          NaN  3.76    Male    Yes   Sat  Dinner     4
162       16.21  2.00  Female     No   Sun  Dinner     3
110       14.00  3.00    Male     No   Sat  Dinner     2
155       29.85  5.14  Female     No   Sun  Dinner     5
146       18.64  1.36  Female     No  Thur   Lunch     3
239       29.03  5.92    Male     No   Sat  Dinner     3
129       22.82  2.18    Male     No  Thur   Lunch     3
------------------------------------
     total_bill   tip     sex smoker   day    time  size  fill_total_bill
113         NaN  2.55    Male     No   Sun  Dinner     2            21.95
207         NaN  3.00    Male    Yes   Sat  Dinner     4            21.95
183         NaN  6.50    Male    Yes   Sun  Dinner     4            21.95
63          NaN  3.76    Male    Yes   S

## Seção 10.4 - Filtragem

Podemos fazer filtragem de grupos. Por exemplo, vamos usar o dataset tips e agrupar as observações por tamanho de grupo de pessoas (coluna 'size'). E aí vamos considerar apenas os grupos que possuem mais de 30 observações na base de dados:

In [10]:
# Carrega o conjunto de dados tips
tips = sns.load_dataset('tips')

# Vamos ver quantos dados temos nesse conjunto:
print('O conjunto original tem tamanho', tips.shape)

# Vamos agrupar por qtd de pessoas na mesa:
print('--------------------------------------------------------------------')
print('Essa é a quantidade de observações no conjunto de dados em função da qtd de pessoas na mesa:')
print(tips['size'].value_counts())

# Agora vamos filtrar os grupos que possuem pelo menos 30 registros:
print('--------------------------------------------------------------------')
print('Essa é a quantidade de observações no conjunto de dados filtrado (pelo menos 30 observações por grupo):')
tips_filtered = tips.groupby('size').filter(lambda x: x['size'].count() >= 30)
print(tips_filtered['size'].value_counts())

O conjunto original tem tamanho (244, 7)
--------------------------------------------------------------------
Essa é a quantidade de observações no conjunto de dados em função da qtd de pessoas na mesa:
2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64
--------------------------------------------------------------------
Essa é a quantidade de observações no conjunto de dados filtrado (pelo menos 30 observações por grupo):
2    156
3     38
4     37
Name: size, dtype: int64


## Seção 10.5 - Objeto pandas.core.groupby.DataFrameGroupBy

Ao realizar um agrupamento podemos visualizar os índices dos registros de cada grupo:

In [11]:
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
print('------------------------------')
print('Dataset completo:')
print(tips_10)

print('------------------------------')
print('Imprimindo dataset agrupado por sex:')
grouped = tips_10.groupby('sex')
print(grouped)

print('------------------------------')
print('Visualizando os índices de cada grupo:')
print(grouped.groups)

------------------------------
Dataset completo:
     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2
------------------------------
Imprimindo dataset agrupado por sex:
------------------------------
Visualizando os índices de cada grupo:
{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}


Ao aplicar uma função em um dataset sem especificar as colunas, o Pandas tentará aplicar em todas as colunas e descartará as colunas onde não é possível:

In [12]:
print('Todas as colunas de tips_10:')
print(tips_10.columns)

print('-------------------------')
print('Aplicando a média chamando função padronizada:')
print(grouped.mean())

print('-------------------------')
print('Aplicando a média chamando função personalizada:')
print(grouped.agg(media_personalizada))

Todas as colunas de tips_10:
Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')
-------------------------
Aplicando a média chamando função padronizada:
        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000
-------------------------
Aplicando a média chamando função personalizada:
        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


Para obter o acesso aos grupos temos que chamar get_group ou iterar o objeto agrupado. Não é possível acessar chamando um índice:

In [13]:
mulheres = grouped.get_group('Female')
print('Apenas mulheres:')
print(mulheres)

print('----------------------------------------------')
print('Iterando em todos os grupos:')
for sex_group in grouped:
    print(sex_group)

Apenas mulheres:
     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2
----------------------------------------------
Iterando em todos os grupos:
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Din

Quando o grupo é iterado, cada objeto é uma tupla, sendo o primeiro elemento o nome do grupo e, o segundo, o dataframe com os registros.

Vamos iterar esse objeto agrupado e ver o conteúdo dos registros:

In [14]:
for item_do_grupo in grouped:
    print('------------------------------------------------------------------------')
    print(f'type(item_do_grupo): {type(item_do_grupo)}\n')
    print(f'len(item_do_grupo): {len(item_do_grupo)}\n')
    print(f'item_do_grupo[0]: {item_do_grupo[0]}\n')
    print(f'type(item_do_grupo[0]): {type(item_do_grupo[0])}\n')
    print(f'item_do_grupo[1]: {item_do_grupo[1]}\n')
    print(f'type(item_do_grupo[1]): {type(item_do_grupo[1])}\n')
    print(f'item_do_grupo: {item_do_grupo}\n')

------------------------------------------------------------------------
type(item_do_grupo): <class 'tuple'>

len(item_do_grupo): 2

item_do_grupo[0]: Male

type(item_do_grupo[0]): <class 'str'>

item_do_grupo[1]:      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2

type(item_do_grupo[1]): <class 'pandas.core.frame.DataFrame'>

item_do_grupo: ('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   

Podemos agrupar por mais de uma variável. Por exemplo, vamos agrupar por sex e time:

In [15]:
bill_sex_time = tips_10.groupby(['sex', 'time'])

group_avg = bill_sex_time.mean()
print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


O print do dataframe mostra ele meio vazio devido a visualização hierárquica dos grupos (as linhas 2 e 4 de 'sex' estão vazias).

Na verdade, se visualizarmos o columns do grupo vamos ver que são colunas apenas o total_bill, tip e size. As outras informações (sex e time) na verdade são os nomes dos índices.

In [16]:
print('Tipo de dados de group_avg:', type(group_avg))

print('As colunas disponíveis:', group_avg.columns)

print('Os índices:', group_avg.index)

Tipo de dados de group_avg: <class 'pandas.core.frame.DataFrame'>
As colunas disponíveis: Index(['total_bill', 'tip', 'size'], dtype='object')
Os índices: MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


Podemos "planificar" esse índice para colocar o dataset com todas as variáveis, mesmo as variáveis que foram para o índice:

In [17]:
group_method = group_avg.reset_index()

print(group_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


Esse mesmo resultado seria atingido usando como parâmetro as_index=False para o método groupby.